In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')


from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
import lightgbm as lgb

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# these two csv contain all the features I've created, I will drop the useless ones later before put into the models
train_X_features = pd.read_csv('/content/drive/MyDrive/train_X_featured.csv')
test_X_featured = pd.read_csv('/content/drive/MyDrive/test_X_featured.csv')
test_X_featured

,region,city,parent_category_name,category_name,param_1,param_2,param_3,price,user_type,title,...,price_log,price_bin,price_to_category_mean,price_to_category_max,price_missing,param_1_missing,param_2_missing,param_3_missing,description_missing,is_train
0,Красноярский край,Красноярск,Личные вещи,"Одежда, обувь, аксессуары",Аксессуары,NaN,NaN,50000.0,Private,Сумка бренд Louis Vuitton,...,10.819798,4,6.065571,1.249375e-04,0,0,1,1,0,0
1,Ярославская область,Ярославль,Личные вещи,Товары для детей и игрушки,Автомобильные кресла,NaN,NaN,21950.0,Company,Автокресло Carmate (Ailebebe) Swing Moon Beige,...,9.996568,4,5.315545,6.271070e-04,0,0,1,1,0,0
2,Тюменская область,Тюмень,Личные вещи,"Одежда, обувь, аксессуары",Мужская одежда,Пиджаки и костюмы,52–54 (XXL),2500.0,Private,Пиджак кожанный,...,7.824446,3,0.303279,6.246875e-06,0,0,0,0,0,0
3,Новосибирская область,Новосибирск,Личные вещи,Детская одежда и обувь,Для девочек,Обувь,32,750.0,Company,Сапоги на слякоть котофей нов,...,6.621406,1,0.076628,2.141938e-06,0,0,0,0,0,0
4,Краснодарский край,Новороссийск,Личные вещи,"Одежда, обувь, аксессуары",Женская одежда,Платья и юбки,42–44 (S),200.0,Private,Юбка,...,5.303305,0,0.024262,4.997500e-07,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
300680,Красноярский край,Красноярск,Личные вещи,"Одежда, обувь, аксессуары",Женская одежда,Обувь,37,1000.0,Private,Сабо,...,6.908755,2,0.121311,2.498750e-06,0,0,0,0,0,0
300681,Свердловская область,Екатеринбург,Бытовая электроника,Телефоны,LG,NaN,NaN,5500.0,Private,Lg Leon h 324,...,8.612685,3,0.182803,1.373281e-05,0,0,1,1,0,0
300682,Тульская область,Тула,Личные вещи,Товары для детей и игрушки,Товары для купания,NaN,NaN,150.0,Private,Ванна детская,...,5.017280,0,0.036325,4.285469e-06,0,0,1,1,0,0
300683,Свердловская область,Екатеринбург,Личные вещи,Детская одежда и обувь,Для девочек,Обувь,24,300.0,Private,"Сапоги эва, размер 24-25",...,5.707110,0,0.030651,8.567751e-07,0,0,0,0,0,0


In [ ]:
def fit_lgb(
        train_X, train_y, test_X, test_y, categorical_features,
        params=None,
        num_boost_round=1000,
        early_stopping_rounds=50,
        eval_period=20,
    ):

    if params is None:
        params = {
            'objective': 'regression',
            'metric': 'rmse',
            'boosting_type': 'gbdt',
            'learning_rate': 0.02,
            'num_leaves': 400,
            'max_depth': 15,

            'bagging_seed': 42,
            'bagging_fraction': 0.8,
            'bagging_freq': 5,
            'feature_fraction': 0.8,
            'lambda_l1': 2.25,
            'lambda_l2': 0.015,

            'verbose': -1
        }

    # label encoding for cat var
    for col in categorical_features:
        train_X[col] = train_X[col].astype('category')
        test_X[col] = test_X[col].astype('category')

    # validation set for early stopping
    train_X_split, val_X_split, train_y_split, val_y_split = train_test_split(
        train_X, train_y, test_size=0.2, random_state=42, shuffle=True
    )
    lgb_train = lgb.Dataset(
        train_X_split, label=train_y_split, categorical_feature=categorical_features
    )
    lgb_val = lgb.Dataset(
        val_X_split, label=val_y_split, categorical_feature=categorical_features,
        reference=lgb_train
    )

    callbacks = [
        lgb.early_stopping(stopping_rounds=early_stopping_rounds),
        lgb.log_evaluation(period=eval_period)
    ]

    model = lgb.train(
        params,
        lgb_train,
        num_boost_round=num_boost_round,
        valid_sets=[lgb_train, lgb_val],
        callbacks=callbacks
    )

    preds = model.predict(test_X, num_iteration=model.best_iteration)
    rmse = mean_squared_error(test_y, preds, squared=False)
    print(f"RMSE: {rmse}")

    return model

In [ ]:
# full features(haven't deleted useless ones)
train_X_features.columns

Index(['region', 'city', 'parent_category_name', 'category_name', 'param_1',
       'param_2', 'param_3', 'price', 'user_type', 'title', 'description',
       'region_city', 'all_category', 'category_param_1',
       'region_category_user', 'city_category_user', 'category_price_mean',
       'category_price_std', 'category_price_max', 'category_price_skew',
       'city_price_mean', 'city_price_std', 'city_price_max',
       'city_price_skew', 'title_length', 'description_length',
       'title_word_count', 'description_word_count', 'title_has_keyword',
       'description_has_keyword', 'title_digit_count',
       'description_digit_count', 'description_newline_count', 'price_log',
       'price_bin', 'price_to_category_mean', 'price_to_category_max',
       'price_missing', 'param_1_missing', 'param_2_missing',
       'param_3_missing', 'description_missing', 'is_train'],
      dtype='object')

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/ML Project/data/train.csv')

train_indices = np.loadtxt('/content/drive/MyDrive/ML Project/data/train_indices.txt', dtype=int)
test_indices = np.loadtxt('/content/drive/MyDrive/ML Project/data/test_indices.txt', dtype=int)

train_df = df.iloc[train_indices]
test_df = df.iloc[test_indices]

In [ ]:
train_df

,item_id,user_id,region,city,parent_category_name,category_name,param_1,param_2,param_3,title,description,price,item_seq_number,activation_date,user_type,image,image_top_1,deal_probability
1501077,771fda0ff8ff,969f1c4acd66,Омская область,Омск,Бытовая электроника,Настольные компьютеры,NaN,NaN,NaN,Продам системник,продам системник на запчасти.,50.0,18,2017-03-28,Private,5f1a4dc40d333d5cd20c9831553a61053e83ffb156f692...,2789.0,0.76786
51680,3c296fc97db9,cf345facfe6b,Башкортостан,Уфа,Хобби и отдых,Спорт и отдых,Зимние виды спорта,NaN,NaN,Продам раздвижные коньки на девочку 32-36 разм,Продам раздвижные коньки на девочку б/у. Разм....,500.0,30,2017-03-16,Private,659f553a15733bbc034bc2ea321a8aaa39feaa3fb1e410...,2645.0,0.00000
26826,13ecb01f22c1,a4ad6dcc8446,Нижегородская область,Дзержинск,Для дома и дачи,Растения,NaN,NaN,NaN,Ромашки и др,1фото-ромашка крупноцветковая/\n2фото-пиретрум...,80.0,25,2017-03-16,Private,7f5b347e054800f384d22a9959792ab6da7b1442931d03...,406.0,0.12311
867845,f3a81005ce3d,a41d7a0cd76f,Челябинская область,Миасс,Личные вещи,Детская одежда и обувь,Для девочек,Верхняя одежда,98-104 см (2-4 года),"Зимний комплект Bilemi p,98","Продам зимний костюм Bilemi p,98 в отличном со...",1500.0,182,2017-03-26,Private,f17dae532061e4a29e25a44fbd90d40ae894c697d636d5...,94.0,0.80323
137360,bc91d5eb9229,19757fb0047c,Башкортостан,Уфа,Для дома и дачи,Бытовая техника,Для кухни,Плиты,NaN,Новая духовка Weissgauff самоочистка сенсор га...,В магазине 22000руб/\n/\nТак же есть новая тех...,18990.0,422,2017-03-16,Shop,ef7789caae6b81233991c9485cf081bcd6530aa4bd9735...,1852.0,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259178,f09ae5daaf01,f9b5910f655a,Ростовская область,Ростов-на-Дону,Услуги,Предложение услуг,"Ремонт, строительство",Сантехника,NaN,Бригада выполняет работы,"Выполняем прокладку наружного трубопровода,мон...",NaN,5,2017-03-21,Private,NaN,NaN,0.20000
1414414,6eab9f61133f,1decc320fb12,Тульская область,Тула,Недвижимость,Квартиры,Сдам,На длительный срок,2,"2-к квартира, 46 м², 2/5 эт.",Сдается уютная 2-комнатная квартира в Пролетар...,13500.0,1211,2017-03-17,Company,19febf47e8ef31b14679e19f276268f6120a0f25b062c8...,2220.0,0.03703
131932,9b5c1f4fca75,196588f9b315,Белгородская область,Алексеевка,Услуги,Предложение услуг,Другое,NaN,NaN,Аренда авто,Автомобили для вашей свадьбы и торжественных ...,700.0,5,2017-03-17,Private,bbee5106cfd831a64aa784f946857088c6cc935333faac...,148.0,0.20000
671155,22c8381d9eab,39b04d8ac30a,Челябинская область,Челябинск,Личные вещи,Товары для детей и игрушки,Детские коляски,NaN,NaN,Коляска Jedo Bartatina Alu Plus Special Edition,Продам коляску 2 в 1 в отличном состоянии посл...,9000.0,21,2017-03-15,Private,23d6b6b7d9903bc236af71dc5abbe93762b5382f4062e8...,1002.0,0.80323


In [ ]:
train_X_features

,region,city,parent_category_name,category_name,param_1,param_2,param_3,price,user_type,title,...,price_log,price_bin,price_to_category_mean,price_to_category_max,price_missing,param_1_missing,param_2_missing,param_3_missing,description_missing,is_train
0,Омская область,Омск,Бытовая электроника,Настольные компьютеры,NaN,NaN,NaN,50.0,Private,Продам системник,...,3.931826,0,0.003888,3.333333e-05,0,1,1,1,0,1
1,Башкортостан,Уфа,Хобби и отдых,Спорт и отдых,Зимние виды спорта,NaN,NaN,500.0,Private,Продам раздвижные коньки на девочку 32-36 разм,...,6.216606,1,0.044637,7.142296e-06,0,0,1,1,0,1
2,Нижегородская область,Дзержинск,Для дома и дачи,Растения,NaN,NaN,NaN,80.0,Private,Ромашки и др,...,4.394449,0,0.034354,7.996002e-05,0,1,1,1,0,1
3,Челябинская область,Миасс,Личные вещи,Детская одежда и обувь,Для девочек,Верхняя одежда,98-104 см (2-4 года),1500.0,Private,"Зимний комплект Bilemi p,98",...,7.313887,2,0.190373,4.990013e-06,0,0,0,0,0,1
4,Башкортостан,Уфа,Для дома и дачи,Бытовая техника,Для кухни,Плиты,NaN,18990.0,Shop,Новая духовка Weissgauff самоочистка сенсор га...,...,9.851720,4,2.354832,3.796861e-03,0,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1202734,Ростовская область,Ростов-на-Дону,Услуги,Предложение услуг,"Ремонт, строительство",Сантехника,NaN,NaN,Private,Бригада выполняет работы,...,NaN,-1,NaN,NaN,1,0,0,1,0,1
1202735,Тульская область,Тула,Недвижимость,Квартиры,Сдам,На длительный срок,2,13500.0,Company,"2-к квартира, 46 м², 2/5 эт.",...,9.510519,4,0.006762,8.365307e-05,0,0,0,0,0,1
1202736,Белгородская область,Алексеевка,Услуги,Предложение услуг,Другое,NaN,NaN,700.0,Private,Аренда авто,...,6.552508,1,0.045174,7.000000e-06,0,0,1,1,0,1
1202737,Челябинская область,Челябинск,Личные вещи,Товары для детей и игрушки,Детские коляски,NaN,NaN,9000.0,Private,Коляска Jedo Bartatina Alu Plus Special Edition,...,9.105091,3,2.100935,2.571282e-04,0,0,1,1,0,1


In [ ]:
test_df

,item_id,user_id,region,city,parent_category_name,category_name,param_1,param_2,param_3,title,description,price,item_seq_number,activation_date,user_type,image,image_top_1,deal_probability
143309,bd45a493569d,6be13830198e,Красноярский край,Красноярск,Личные вещи,"Одежда, обувь, аксессуары",Аксессуары,NaN,NaN,Сумка бренд Louis Vuitton,"Сумка коллекционная, в хорошем состоянии! Нату...",50000.0,99,2017-03-19,Private,3e6502f1657e3f256be696fd82feb455ddc82345e3577e...,256.0,0.00000
771572,321127a6a33b,98b93e9aaf33,Ярославская область,Ярославль,Личные вещи,Товары для детей и игрушки,Автомобильные кресла,NaN,NaN,Автокресло Carmate (Ailebebe) Swing Moon Beige,"Посмотреть, примерить и купить автокресло можн...",21950.0,417,2017-03-18,Company,9270dff29b16082e5a01d35a418cd6d525b6618c33fc58...,796.0,0.00000
1378472,d976d5211526,7246f2b892e9,Тюменская область,Тюмень,Личные вещи,"Одежда, обувь, аксессуары",Мужская одежда,Пиджаки и костюмы,52–54 (XXL),Пиджак кожанный,"Состояние на 4, торг",2500.0,19,2017-03-19,Private,bbe390ab0a1ed2cb97c4ed064524e9809825dfeba713c0...,655.0,0.00000
846158,c046d8685345,3ceed1f36bec,Новосибирская область,Новосибирск,Личные вещи,Детская одежда и обувь,Для девочек,Обувь,32,Сапоги на слякоть котофей нов,Сапожки из невесомого материала ЭВА. Манжет из...,750.0,590,2017-03-20,Company,e528c5df69dbc8d6073e9b617b174bec2405cf7e6e3689...,91.0,0.00000
84567,98a8ba1ff4c4,feb28eeb11a9,Краснодарский край,Новороссийск,Личные вещи,"Одежда, обувь, аксессуары",Женская одежда,Платья и юбки,42–44 (S),Юбка,"Юбка женская, размер 42-44, цвет черный, летня...",200.0,27,2017-03-19,Private,ce15238bc8d1d093b79ade25f1346be1acfc7d9570ec1b...,81.0,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
186583,07a88b2f9ffc,868bce3fbbb2,Красноярский край,Красноярск,Личные вещи,"Одежда, обувь, аксессуары",Женская одежда,Обувь,37,Сабо,"Состояние хорошее,замша натуральная.",1000.0,19,2017-03-17,Private,3d1962234a5f72ea12dae871d7af1f7f5016094ef92a9d...,445.0,0.00000
540298,a7b7029023e4,3217919a362d,Свердловская область,Екатеринбург,Бытовая электроника,Телефоны,LG,NaN,NaN,Lg Leon h 324,"Документы имеются.В хорошем рабочем состоянии,...",5500.0,1,2017-03-26,Private,18210297b0e61f0aed44722dd7a2e2920cd9b8ee404e47...,2942.0,0.07122
360953,efecf8d66b20,cdb3b17927a8,Тульская область,Тула,Личные вещи,Товары для детей и игрушки,Товары для купания,NaN,NaN,Ванна детская,Ванночка детская. Цвет Лайма,150.0,46,2017-03-25,Private,7542a8ce604ab56601027660a4f181d66d642e8627a492...,1019.0,0.00000
383137,b4e141ae7703,845b032118f6,Свердловская область,Екатеринбург,Личные вещи,Детская одежда и обувь,Для девочек,Обувь,24,"Сапоги эва, размер 24-25",Внутри съемный носок,300.0,297,2017-03-18,Private,6b08256fcd0c6f81760b97c8b5a02f711919870333d9b7...,89.0,0.80323


In [ ]:
test_X_featured

,region,city,parent_category_name,category_name,param_1,param_2,param_3,price,user_type,title,...,price_log,price_bin,price_to_category_mean,price_to_category_max,price_missing,param_1_missing,param_2_missing,param_3_missing,description_missing,is_train
0,Красноярский край,Красноярск,Личные вещи,"Одежда, обувь, аксессуары",Аксессуары,NaN,NaN,50000.0,Private,Сумка бренд Louis Vuitton,...,10.819798,4,6.065571,1.249375e-04,0,0,1,1,0,0
1,Ярославская область,Ярославль,Личные вещи,Товары для детей и игрушки,Автомобильные кресла,NaN,NaN,21950.0,Company,Автокресло Carmate (Ailebebe) Swing Moon Beige,...,9.996568,4,5.315545,6.271070e-04,0,0,1,1,0,0
2,Тюменская область,Тюмень,Личные вещи,"Одежда, обувь, аксессуары",Мужская одежда,Пиджаки и костюмы,52–54 (XXL),2500.0,Private,Пиджак кожанный,...,7.824446,3,0.303279,6.246875e-06,0,0,0,0,0,0
3,Новосибирская область,Новосибирск,Личные вещи,Детская одежда и обувь,Для девочек,Обувь,32,750.0,Company,Сапоги на слякоть котофей нов,...,6.621406,1,0.076628,2.141938e-06,0,0,0,0,0,0
4,Краснодарский край,Новороссийск,Личные вещи,"Одежда, обувь, аксессуары",Женская одежда,Платья и юбки,42–44 (S),200.0,Private,Юбка,...,5.303305,0,0.024262,4.997500e-07,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
300680,Красноярский край,Красноярск,Личные вещи,"Одежда, обувь, аксессуары",Женская одежда,Обувь,37,1000.0,Private,Сабо,...,6.908755,2,0.121311,2.498750e-06,0,0,0,0,0,0
300681,Свердловская область,Екатеринбург,Бытовая электроника,Телефоны,LG,NaN,NaN,5500.0,Private,Lg Leon h 324,...,8.612685,3,0.182803,1.373281e-05,0,0,1,1,0,0
300682,Тульская область,Тула,Личные вещи,Товары для детей и игрушки,Товары для купания,NaN,NaN,150.0,Private,Ванна детская,...,5.017280,0,0.036325,4.285469e-06,0,0,1,1,0,0
300683,Свердловская область,Екатеринбург,Личные вещи,Детская одежда и обувь,Для девочек,Обувь,24,300.0,Private,"Сапоги эва, размер 24-25",...,5.707110,0,0.030651,8.567751e-07,0,0,0,0,0,0


In [ ]:
train_df.columns

Index(['item_id', 'user_id', 'region', 'city', 'parent_category_name',
       'category_name', 'param_1', 'param_2', 'param_3', 'title',
       'description', 'price', 'item_seq_number', 'activation_date',
       'user_type', 'image', 'image_top_1', 'deal_probability'],
      dtype='object')

In [ ]:
train_X_features.columns

Index(['region', 'city', 'parent_category_name', 'category_name', 'param_1',
       'param_2', 'param_3', 'price', 'user_type', 'title', 'description',
       'region_city', 'all_category', 'category_param_1',
       'region_category_user', 'city_category_user', 'category_price_mean',
       'category_price_std', 'category_price_max', 'category_price_skew',
       'city_price_mean', 'city_price_std', 'city_price_max',
       'city_price_skew', 'title_length', 'description_length',
       'title_word_count', 'description_word_count', 'title_has_keyword',
       'description_has_keyword', 'title_digit_count',
       'description_digit_count', 'description_newline_count', 'price_log',
       'price_bin', 'price_to_category_mean', 'price_to_category_max',
       'price_missing', 'param_1_missing', 'param_2_missing',
       'param_3_missing', 'description_missing', 'is_train'],
      dtype='object')

In [ ]:
# add text image columns back
missing_columns = [col for col in train_df.columns if col not in train_X_features.columns]
missing_columns

['item_id',
 'user_id',
 'item_seq_number',
 'activation_date',
 'image',
 'image_top_1',
 'deal_probability']

In [ ]:
additional_original_features = train_df[missing_columns].reset_index(drop=True)
train_X_y_features_v1 = pd.concat([train_X_features, additional_original_features], axis=1)
train_X_y_features_v1

,region,city,parent_category_name,category_name,param_1,param_2,param_3,price,user_type,title,...,param_3_missing,description_missing,is_train,item_id,user_id,item_seq_number,activation_date,image,image_top_1,deal_probability
0,Омская область,Омск,Бытовая электроника,Настольные компьютеры,NaN,NaN,NaN,50.0,Private,Продам системник,...,1,0,1,771fda0ff8ff,969f1c4acd66,18,2017-03-28,5f1a4dc40d333d5cd20c9831553a61053e83ffb156f692...,2789.0,0.76786
1,Башкортостан,Уфа,Хобби и отдых,Спорт и отдых,Зимние виды спорта,NaN,NaN,500.0,Private,Продам раздвижные коньки на девочку 32-36 разм,...,1,0,1,3c296fc97db9,cf345facfe6b,30,2017-03-16,659f553a15733bbc034bc2ea321a8aaa39feaa3fb1e410...,2645.0,0.00000
2,Нижегородская область,Дзержинск,Для дома и дачи,Растения,NaN,NaN,NaN,80.0,Private,Ромашки и др,...,1,0,1,13ecb01f22c1,a4ad6dcc8446,25,2017-03-16,7f5b347e054800f384d22a9959792ab6da7b1442931d03...,406.0,0.12311
3,Челябинская область,Миасс,Личные вещи,Детская одежда и обувь,Для девочек,Верхняя одежда,98-104 см (2-4 года),1500.0,Private,"Зимний комплект Bilemi p,98",...,0,0,1,f3a81005ce3d,a41d7a0cd76f,182,2017-03-26,f17dae532061e4a29e25a44fbd90d40ae894c697d636d5...,94.0,0.80323
4,Башкортостан,Уфа,Для дома и дачи,Бытовая техника,Для кухни,Плиты,NaN,18990.0,Shop,Новая духовка Weissgauff самоочистка сенсор га...,...,1,0,1,bc91d5eb9229,19757fb0047c,422,2017-03-16,ef7789caae6b81233991c9485cf081bcd6530aa4bd9735...,1852.0,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1202734,Ростовская область,Ростов-на-Дону,Услуги,Предложение услуг,"Ремонт, строительство",Сантехника,NaN,NaN,Private,Бригада выполняет работы,...,1,0,1,f09ae5daaf01,f9b5910f655a,5,2017-03-21,NaN,NaN,0.20000
1202735,Тульская область,Тула,Недвижимость,Квартиры,Сдам,На длительный срок,2,13500.0,Company,"2-к квартира, 46 м², 2/5 эт.",...,0,0,1,6eab9f61133f,1decc320fb12,1211,2017-03-17,19febf47e8ef31b14679e19f276268f6120a0f25b062c8...,2220.0,0.03703
1202736,Белгородская область,Алексеевка,Услуги,Предложение услуг,Другое,NaN,NaN,700.0,Private,Аренда авто,...,1,0,1,9b5c1f4fca75,196588f9b315,5,2017-03-17,bbee5106cfd831a64aa784f946857088c6cc935333faac...,148.0,0.20000
1202737,Челябинская область,Челябинск,Личные вещи,Товары для детей и игрушки,Детские коляски,NaN,NaN,9000.0,Private,Коляска Jedo Bartatina Alu Plus Special Edition,...,1,0,1,22c8381d9eab,39b04d8ac30a,21,2017-03-15,23d6b6b7d9903bc236af71dc5abbe93762b5382f4062e8...,1002.0,0.80323


In [ ]:
train_X_y_features_v1.columns

Index(['region', 'city', 'parent_category_name', 'category_name', 'param_1',
       'param_2', 'param_3', 'price', 'user_type', 'title', 'description',
       'region_city', 'all_category', 'category_param_1',
       'region_category_user', 'city_category_user', 'category_price_mean',
       'category_price_std', 'category_price_max', 'category_price_skew',
       'city_price_mean', 'city_price_std', 'city_price_max',
       'city_price_skew', 'title_length', 'description_length',
       'title_word_count', 'description_word_count', 'title_has_keyword',
       'description_has_keyword', 'title_digit_count',
       'description_digit_count', 'description_newline_count', 'price_log',
       'price_bin', 'price_to_category_mean', 'price_to_category_max',
       'price_missing', 'param_1_missing', 'param_2_missing',
       'param_3_missing', 'description_missing', 'is_train', 'item_id',
       'user_id', 'item_seq_number', 'activation_date', 'image', 'image_top_1',
       'deal_probability

In [ ]:
additional_original_features_test

,item_id,user_id,item_seq_number,activation_date,image,image_top_1,deal_probability
143309,bd45a493569d,6be13830198e,99,2017-03-19,3e6502f1657e3f256be696fd82feb455ddc82345e3577e...,256.0,0.00000
771572,321127a6a33b,98b93e9aaf33,417,2017-03-18,9270dff29b16082e5a01d35a418cd6d525b6618c33fc58...,796.0,0.00000
1378472,d976d5211526,7246f2b892e9,19,2017-03-19,bbe390ab0a1ed2cb97c4ed064524e9809825dfeba713c0...,655.0,0.00000
846158,c046d8685345,3ceed1f36bec,590,2017-03-20,e528c5df69dbc8d6073e9b617b174bec2405cf7e6e3689...,91.0,0.00000
84567,98a8ba1ff4c4,feb28eeb11a9,27,2017-03-19,ce15238bc8d1d093b79ade25f1346be1acfc7d9570ec1b...,81.0,0.00000
...,...,...,...,...,...,...,...
186583,07a88b2f9ffc,868bce3fbbb2,19,2017-03-17,3d1962234a5f72ea12dae871d7af1f7f5016094ef92a9d...,445.0,0.00000
540298,a7b7029023e4,3217919a362d,1,2017-03-26,18210297b0e61f0aed44722dd7a2e2920cd9b8ee404e47...,2942.0,0.07122
360953,efecf8d66b20,cdb3b17927a8,46,2017-03-25,7542a8ce604ab56601027660a4f181d66d642e8627a492...,1019.0,0.00000
383137,b4e141ae7703,845b032118f6,297,2017-03-18,6b08256fcd0c6f81760b97c8b5a02f711919870333d9b7...,89.0,0.80323


In [ ]:
additional_original_features_test = test_df[missing_columns].reset_index(drop=True)
test_X_y_features_v1 = pd.concat([test_X_featured, additional_original_features_test], axis=1)
test_X_y_features_v1

,region,city,parent_category_name,category_name,param_1,param_2,param_3,price,user_type,title,...,param_3_missing,description_missing,is_train,item_id,user_id,item_seq_number,activation_date,image,image_top_1,deal_probability
0,Красноярский край,Красноярск,Личные вещи,"Одежда, обувь, аксессуары",Аксессуары,NaN,NaN,50000.0,Private,Сумка бренд Louis Vuitton,...,1,0,0,bd45a493569d,6be13830198e,99,2017-03-19,3e6502f1657e3f256be696fd82feb455ddc82345e3577e...,256.0,0.00000
1,Ярославская область,Ярославль,Личные вещи,Товары для детей и игрушки,Автомобильные кресла,NaN,NaN,21950.0,Company,Автокресло Carmate (Ailebebe) Swing Moon Beige,...,1,0,0,321127a6a33b,98b93e9aaf33,417,2017-03-18,9270dff29b16082e5a01d35a418cd6d525b6618c33fc58...,796.0,0.00000
2,Тюменская область,Тюмень,Личные вещи,"Одежда, обувь, аксессуары",Мужская одежда,Пиджаки и костюмы,52–54 (XXL),2500.0,Private,Пиджак кожанный,...,0,0,0,d976d5211526,7246f2b892e9,19,2017-03-19,bbe390ab0a1ed2cb97c4ed064524e9809825dfeba713c0...,655.0,0.00000
3,Новосибирская область,Новосибирск,Личные вещи,Детская одежда и обувь,Для девочек,Обувь,32,750.0,Company,Сапоги на слякоть котофей нов,...,0,0,0,c046d8685345,3ceed1f36bec,590,2017-03-20,e528c5df69dbc8d6073e9b617b174bec2405cf7e6e3689...,91.0,0.00000
4,Краснодарский край,Новороссийск,Личные вещи,"Одежда, обувь, аксессуары",Женская одежда,Платья и юбки,42–44 (S),200.0,Private,Юбка,...,0,0,0,98a8ba1ff4c4,feb28eeb11a9,27,2017-03-19,ce15238bc8d1d093b79ade25f1346be1acfc7d9570ec1b...,81.0,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
300680,Красноярский край,Красноярск,Личные вещи,"Одежда, обувь, аксессуары",Женская одежда,Обувь,37,1000.0,Private,Сабо,...,0,0,0,07a88b2f9ffc,868bce3fbbb2,19,2017-03-17,3d1962234a5f72ea12dae871d7af1f7f5016094ef92a9d...,445.0,0.00000
300681,Свердловская область,Екатеринбург,Бытовая электроника,Телефоны,LG,NaN,NaN,5500.0,Private,Lg Leon h 324,...,1,0,0,a7b7029023e4,3217919a362d,1,2017-03-26,18210297b0e61f0aed44722dd7a2e2920cd9b8ee404e47...,2942.0,0.07122
300682,Тульская область,Тула,Личные вещи,Товары для детей и игрушки,Товары для купания,NaN,NaN,150.0,Private,Ванна детская,...,1,0,0,efecf8d66b20,cdb3b17927a8,46,2017-03-25,7542a8ce604ab56601027660a4f181d66d642e8627a492...,1019.0,0.00000
300683,Свердловская область,Екатеринбург,Личные вещи,Детская одежда и обувь,Для девочек,Обувь,24,300.0,Private,"Сапоги эва, размер 24-25",...,0,0,0,b4e141ae7703,845b032118f6,297,2017-03-18,6b08256fcd0c6f81760b97c8b5a02f711919870333d9b7...,89.0,0.80323


In [ ]:
test_X_y_features_v1.columns

Index(['region', 'city', 'parent_category_name', 'category_name', 'param_1',
       'param_2', 'param_3', 'price', 'user_type', 'title', 'description',
       'region_city', 'all_category', 'category_param_1',
       'region_category_user', 'city_category_user', 'category_price_mean',
       'category_price_std', 'category_price_max', 'category_price_skew',
       'city_price_mean', 'city_price_std', 'city_price_max',
       'city_price_skew', 'title_length', 'description_length',
       'title_word_count', 'description_word_count', 'title_has_keyword',
       'description_has_keyword', 'title_digit_count',
       'description_digit_count', 'description_newline_count', 'price_log',
       'price_bin', 'price_to_category_mean', 'price_to_category_max',
       'price_missing', 'param_1_missing', 'param_2_missing',
       'param_3_missing', 'description_missing', 'is_train', 'item_id',
       'user_id', 'item_seq_number', 'activation_date', 'image', 'image_top_1',
       'deal_probability

In [ ]:
train_X_y_features_v1 = train_X_y_features_v1.drop(['is_train','param_1_missing', 'param_2_missing', 'param_3_missing','price_missing','category_price_max','city_price_std'],axis = 1)
test_X_y_features_v1 = test_X_y_features_v1.drop(['is_train','param_1_missing', 'param_2_missing', 'param_3_missing','price_missing','category_price_max','city_price_std'],axis = 1)


In [ ]:
# original data(with text and image), and with newly created useful features.
train_X_y_features_v1.columns

Index(['region', 'city', 'parent_category_name', 'category_name', 'param_1',
       'param_2', 'param_3', 'price', 'user_type', 'title', 'description',
       'region_city', 'all_category', 'category_param_1',
       'region_category_user', 'city_category_user', 'category_price_mean',
       'category_price_std', 'category_price_skew', 'city_price_mean',
       'city_price_max', 'city_price_skew', 'title_length',
       'description_length', 'title_word_count', 'description_word_count',
       'title_has_keyword', 'description_has_keyword', 'title_digit_count',
       'description_digit_count', 'description_newline_count', 'price_log',
       'price_bin', 'price_to_category_mean', 'price_to_category_max',
       'description_missing', 'item_id', 'user_id', 'item_seq_number',
       'activation_date', 'image', 'image_top_1', 'deal_probability'],
      dtype='object')

In [ ]:
train_X_y_features_v1.to_csv('train_X_y_features_v1.csv',index=False)
test_X_y_features_v1.to_csv('test_X_y_features_v1.csv',index=False)
